**parallelize** 

parallelizing in PySpark means splitting a task or dataset into smaller parts and processing them simultaneously on multiple machines or cores to speed up computation.

In PySpark, this is done using the `parallelize()` method, which takes a regular Python collection (like a list or a set) and converts it into an RDD (Resilient Distributed Dataset). This RDD can then be processed in parallel across a cluster of machines.

For example, if you have a large dataset (like a list of numbers), instead of processing the entire dataset sequentially, PySpark will break it into smaller chunks and work on them in parallel. This results in faster processing, especially when dealing with large datasets.

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySparkExample") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/20 13:57:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/20 13:57:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
words = ("big","Data","Is","SUPER","Interesting","BIG","data","IS","A","Trending","technology")

In [3]:
words_rdd = spark.sparkContext.parallelize(words)

In [4]:
words_normalized = words_rdd.map(lambda x: x.lower())

In [5]:
words_normalized.collect()

['big',
 'data',
 'is',
 'super',
 'interesting',
 'big',
 'data',
 'is',
 'a',
 'trending',
 'technology']

In [6]:
mapped_words = words_normalized.map(lambda x:(x,1))

In [7]:
aggregated_result = mapped_words.reduceByKey(lambda x,y: x+y)

In [8]:
aggregated_result.collect()

[('interesting', 1),
 ('trending', 1),
 ('technology', 1),
 ('a', 1),
 ('data', 2),
 ('super', 1),
 ('is', 2),
 ('big', 2)]

**Chaining functions together in PySpark** 

Which means applying multiple operations or transformations one after another to an RDD or DataFrame without having to assign intermediate results to separate variables.

In PySpark, operations like `map()`, `filter()`, `reduce()`, or `select()` can be chained together to perform a series of transformations or actions on your data. Chaining makes the code more concise and readable, and it allows Spark to optimize the execution plan by combining these operations into a single pipeline.

By chaining the .map(), .filter(), and .collect() methods together with the `\ (backslash)`, we avoid creating intermediate variables and directly apply the transformations in sequence.

**Why Chaining is Useful**

*   **Conciseness:** It reduces the need for intermediate variables and makes your code more compact.
*   **Readability:** It makes it clear that a series of transformations are being applied to the data
*   **Optimization:** Spark can optimize the sequence of operations in a single execution plan, potentially making the computation more efficient.

In summary, chaining in PySpark allows you to apply multiple transformations and actions in sequence without intermediate steps, which results in cleaner, more efficient code.

In [9]:
result = spark.sparkContext.parallelize(words)  \
.map(lambda x: x.lower())   \
.map(lambda x:(x,1))    \
.reduceByKey(lambda x,y: x+y)

In [10]:
result.collect()

[('interesting', 1),
 ('trending', 1),
 ('technology', 1),
 ('a', 1),
 ('data', 2),
 ('super', 1),
 ('is', 2),
 ('big', 2)]

**How will you check that how many partitions your rdd has?**

To check how many partitions an RDD has in PySpark, you can use the `getNumPartitions()` method.

This method returns the number of partitions in the RDD, which is useful for understanding how the data is distributed across the Spark cluster.


**Why is this useful?**

*   Knowing the number of partitions is helpful for optimizing performance in Spark. Too few partitions may lead to uneven workload distribution, while too many partitions could lead to overhead in managing them.

*   You can also repartition your RDD to control the number of partitions using operations like     `repartition()` or `coalesce()` to optimize the performance based on your cluster's configuration.

In [11]:
words_rdd.getNumPartitions()

8

The value returned by `getNumPartitions()` in PySpark depends on several factors, such as how the RDD was created and how the data is distributed across the cluster or local machine. The number of partitions is influenced by the following:

**1. Number of Partitions Specified During RDD Creation**

When you create an RDD, you can specify the number of partitions explicitly. For example, using sc.`parallelize()`, you can set the number of partitions to control how the data will be distributed.

Example:-

In [12]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 3)
print(rdd.getNumPartitions())  # Output: 3

3


In this case, you specified 3 partitions, so the result will be 3.

**2. Default Number of Partitions (When Not Specified)**


If you don't specify the number of partitions during RDD creation (e.g., using sc.parallelize()), Spark uses the default number of partitions, which is determined by:

*   **For Local Mode:** The default number of partitions is typically the number of available cores on the machine.
    *   Example: If you run Spark on a machine with 8 cores, the default parallelism will be 8, and sc.parallelize(data) will create an RDD with 8 partitions by default.

*   **For Cluster Mode:** The default number of partitions will be based on the total number of cores across all nodes in the cluster.
    *   Example: If you have 10 nodes, each with 8 cores, the default parallelism might be 80.

This default number of partitions can be retrieved using `spark.sparkContext.defaultParallelism`.

Example:-

In [13]:
print(spark.sparkContext.defaultParallelism)  # Output might be 8 (if running locally with 8 cores)

8


**3.Data Source and File Size**

When you're reading data from an external source (e.g., HDFS, S3, or local files), the number of partitions can be influenced by:

*   The number of input files and their size.

*   The default partitioning behavior of Spark when reading the data (e.g., how it splits files into partitions).

For example:

*   If you load a large text file, Spark will split the file into partitions based on block size (typically 128 MB per partition in HDFS).
*   If you load the small, Spark may split create a partition & distribute the data based on spark.`sparkContext.defaultMinPartitions`

In [14]:
spark.sparkContext.defaultMinPartitions

2

In [15]:
base_rdd = spark.sparkContext.textFile("/Users/sugumarsrinivasan/Documents/data/input_data.txt")

In [16]:
""" if the file size is less than 128 MB(i.e. default block size in HDFS), 
then Number of partitions are equal to spark.sparkContext.deafultMinPartitions set in the cluster."""
base_rdd.getNumPartitions() 

2